In [1]:
import os
import cv2

# Define the base directory where your folders are stored
base_dir = r"C:\Users\91790\OneDrive\Desktop\BHAVATHARINI\OCULAR DISEASE\Ocular Dataset"

# List of folder names to read images from
folders = ["Cataract_median_equalized", "Diabetic Retinopathy_equalized", "Normal_median_equalized", "Glaucoma_median_equalized"]

# Dictionary to store images from each folder
images_by_folder = {}

# Function to read images from a specified folder and store in the dictionary
def read_images(folder_name):
    # Path to the specific folder
    folder_path = os.path.join(base_dir, folder_name)

    # Get all image file names in the folder
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # List to store images from this folder
    images = []

    # Read and store each image in the list
    for image_file in image_files:
        # Read the image in color
        img_path = os.path.join(folder_path, image_file)
        image = cv2.imread(img_path, cv2.IMREAD_COLOR)

        if image is not None:
            images.append(image)
        else:
            print(f"Could not read the image: {image_file}")

    # Store the list of images in the dictionary with the folder name as the key
    images_by_folder[folder_name] = images

# Loop through each folder and read the images
for folder in folders:
    print(f"Reading images from folder: {folder}")
    read_images(folder)


Reading images from folder: Cataract_median_equalized
Reading images from folder: Diabetic Retinopathy_equalized
Reading images from folder: Normal_median_equalized
Reading images from folder: Glaucoma_median_equalized


In [2]:
!pip install scikit-image

In [3]:
import os
import cv2
import numpy as np
from skimage.feature import hog
from skimage import exposure

# Base directory where subfolders are located
base_dir = r"C:\Users\91790\OneDrive\Desktop\BHAVATHARINI\OCULAR DISEASE\Ocular Dataset"

# List of specific subfolders to process
specific_subfolders = ["Cataract_median_equalized", "Diabetic Retinopathy_equalized", "Normal_median_equalized", "Glaucoma_median_equalized"]

# Filter subfolders to process only specific ones
subfolders = [f for f in specific_subfolders if os.path.isdir(os.path.join(base_dir, f))]

# Dictionary to store features for specific subfolders
features_by_folder = {}

# Function to extract HOG features from a grayscale image
def extract_hog_features(image):
    # Extract HOG features and a visualization image
    hog_features, hog_image = hog(image, visualize=True, pixels_per_cell=(16, 16), cells_per_block=(1, 1), feature_vector=True)
    
    # Rescale the HOG image for visualization (optional)
    hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))
    
    return hog_features, hog_image_rescaled

# Function to read images from a folder and extract HOG features
def read_and_extract_features(folder_path):
    # List all image files in the folder
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # Store HOG features from this folder
    hog_features_list = []

    # Loop through each image and extract HOG features
    for image_file in image_files:
        img_path = os.path.join(folder_path, image_file)
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        if image is not None:
            # Extract HOG features
            hog_features, _ = extract_hog_features(image)

            # Append the HOG features to the list
            hog_features_list.append(hog_features)
    
    return hog_features_list

# Loop through each subfolder to extract HOG features
for subfolder in subfolders:
    print(f"Processing subfolder: {subfolder}")
    folder_path = os.path.join(base_dir, subfolder)

    # Extract features from the current subfolder
    features = read_and_extract_features(folder_path)

    # Store the extracted features in the dictionary
    features_by_folder[subfolder] = features

    # Provide feedback on the number of images processed
    print(f"Extracted HOG features from {len(features)} images in '{subfolder}'")


Processing subfolder: Cataract_median_equalized
Extracted HOG features from 1032 images in 'Cataract_median_equalized'
Processing subfolder: Diabetic Retinopathy_equalized
Extracted HOG features from 1098 images in 'Diabetic Retinopathy_equalized'
Processing subfolder: Normal_median_equalized
Extracted HOG features from 1074 images in 'Normal_median_equalized'
Processing subfolder: Glaucoma_median_equalized
Extracted HOG features from 1007 images in 'Glaucoma_median_equalized'


In [5]:
import os
import cv2
import numpy as np

# Base directory where subfolders are located
base_dir = r"C:\Users\91790\OneDrive\Desktop\BHAVATHARINI\OCULAR DISEASE\Ocular Dataset"

# List of specific subfolders to process
specific_subfolders = ["Cataract_median_equalized", "Diabetic Retinopathy_equalized", "Normal_median_equalized", "Glaucoma_median_equalized"]
# Dictionary to store segmented images for each subfolder
segmented_by_folder = {}

# Function to apply Otsu's Thresholding for segmentation
def otsu_threshold_segmentation(image):
    # Apply Gaussian Blur to reduce noise
    blurred_image = cv2.GaussianBlur(image, (5, 5), 0)

    # Apply Otsu's Thresholding to segment the image
    _, segmented_image = cv2.threshold(
        blurred_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU
    )

    return segmented_image

# Function to read images from a folder and apply segmentation
def read_and_segment_images(folder_path):
    # List all image files in the folder
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # List to store segmented images
    segmented_images = []

    # Loop through each image and apply segmentation
    for image_file in image_files:
        img_path = os.path.join(folder_path, image_file)
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        if image is not None:
            # Apply Otsu's Thresholding for segmentation
            segmented_image = otsu_threshold_segmentation(image)

            # Store the segmented image in the list
            segmented_images.append({
                'original_image': image,
                'segmented_image': segmented_image,
            })

    return segmented_images

# Loop through each specified subfolder to extract and store segmented images
for subfolder in specific_subfolders:
    print(f"Processing segmentation for subfolder: {subfolder}")
    folder_path = os.path.join(base_dir, subfolder)

    # Apply segmentation and store in memory
    segmented_images = read_and_segment_images(folder_path)

    # Store the segmented images in a dictionary
    segmented_by_folder[subfolder] = segmented_images

    # Provide feedback on the number of segmented images
    print(f"Segmented {len(segmented_images)} images in '{subfolder}'")

# Now `segmented_by_folder` contains all segmented images in memory, which can be used for further processing.


Processing segmentation for subfolder: Cataract_median_equalized
Segmented 1032 images in 'Cataract_median_equalized'
Processing segmentation for subfolder: Diabetic Retinopathy_equalized
Segmented 1098 images in 'Diabetic Retinopathy_equalized'
Processing segmentation for subfolder: Normal_median_equalized
Segmented 1074 images in 'Normal_median_equalized'
Processing segmentation for subfolder: Glaucoma_median_equalized
Segmented 1007 images in 'Glaucoma_median_equalized'


In [6]:
!pip install tensorflow

In [9]:
import os
import shutil  # This will help to copy files and folders
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Base directory for the dataset
base_dir = r"C:\Users\91790\OneDrive\Desktop\BHAVATHARINI\OCULAR DISEASE\Ocular Dataset"

# List of specific subfolders to process
target_subfolders = ["Cataract_median_equalized", "Diabetic Retinopathy_equalized", "Normal_median_equalized", "Glaucoma_median_equalized"]

# Create a new temporary base directory
temp_base_dir = r"C:\Users\91790\OneDrive\Desktop\BHAVATHARINI\OCULAR DISEASE\Temp"

# Ensure the temporary base directory is empty and exists
if os.path.exists(temp_base_dir):
    shutil.rmtree(temp_base_dir)  # Clear if exists
os.makedirs(temp_base_dir, exist_ok=True)  # Create a fresh folder

# Copy the specified subfolders to the temporary base directory
for subfolder in target_subfolders:
    src_path = os.path.join(base_dir, subfolder)
    dest_path = os.path.join(temp_base_dir, subfolder)

    # Copy the entire subfolder if it exists
    if os.path.isdir(src_path):
        shutil.copytree(src_path, dest_path)

# Prepare the data generator for data augmentation
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Training dataset from specified subfolders
train_generator = datagen.flow_from_directory(
    temp_base_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

# Validation dataset from specified subfolders
val_generator = datagen.flow_from_directory(
    temp_base_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Define a simple CNN model
cnn_model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_generator.num_classes, activation='softmax')  # Output layer for categorical classification
])

# Compile the model
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
cnn_model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

# Evaluate the model
loss, accuracy = cnn_model.evaluate(val_generator)
print(f"Validation accuracy: {accuracy:.2f}")


Found 3371 images belonging to 4 classes.
Found 840 images belonging to 4 classes.
Epoch 1/10
106/106 [==============================] - 83s 765ms/step - loss: 1.0176 - accuracy: 0.5268 - val_loss: 0.8416 - val_accuracy: 0.5690
Epoch 2/10
106/106 [==============================] - 60s 564ms/step - loss: 0.7081 - accuracy: 0.6811 - val_loss: 0.7414 - val_accuracy: 0.6250
Epoch 3/10
106/106 [==============================] - 60s 567ms/step - loss: 0.5995 - accuracy: 0.7404 - val_loss: 0.5826 - val_accuracy: 0.7345
Epoch 4/10
106/106 [==============================] - 60s 566ms/step - loss: 0.5167 - accuracy: 0.7820 - val_loss: 0.5938 - val_accuracy: 0.7274
Epoch 5/10
106/106 [==============================] - 60s 566ms/step - loss: 0.4641 - accuracy: 0.8015 - val_loss: 0.5791 - val_accuracy: 0.7429
Epoch 6/10
106/106 [==============================] - 60s 564ms/step - loss: 0.4123 - accuracy: 0.8300 - val_loss: 0.6366 - val_accuracy: 0.7298
Epoch 7/10
106/106 [===========================

In [11]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Temporary base directory for the dataset
temp_base_dir = r"C:\Users\91790\OneDrive\Desktop\BHAVATHARINI\OCULAR DISEASE\Temp"

# Prepare the data generator for data augmentation
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Training dataset from specified subfolders
train_generator = datagen.flow_from_directory(
    temp_base_dir,
    target_size=(227, 227),  # AlexNet uses 227x227 images
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

# Validation dataset from specified subfolders
val_generator = datagen.flow_from_directory(
    temp_base_dir,
    target_size=(227, 227),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Define an AlexNet architecture
alexnet_model = models.Sequential([
    layers.Conv2D(96, (11, 11), strides=(4, 4), activation='relu', input_shape=(227, 227, 3)),  # Conv1
    layers.MaxPooling2D((3, 3), strides=(2, 2)),  # Pool1
    layers.Conv2D(256, (5, 5), padding='same', activation='relu'),  # Conv2
    layers.MaxPooling2D((3, 3), strides=(2, 2)),  # Pool2
    layers.Conv2D(384, (3, 3), padding='same', activation='relu'),  # Conv3
    layers.Conv2D(384, (3, 3), padding='same', activation='relu'),  # Conv4
    layers.Conv2D(256, (3, 3), padding='same', activation='relu'),  # Conv5
    layers.MaxPooling2D((3, 3), strides=(2, 2)),  # Pool3
    layers.Flatten(),  # Flattening for fully connected layers
    layers.Dense(4096, activation='relu'),  # FC6
    layers.Dropout(0.5),  # Dropout for regularization
    layers.Dense(4096, activation='relu'),  # FC7
    layers.Dropout(0.5),
    layers.Dense(train_generator.num_classes, activation='softmax')  # Output layer for categorical classification
])

# Compile the model
alexnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
alexnet_model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

# Evaluate the model
loss, accuracy = alexnet_model.evaluate(val_generator)
print(f"Validation accuracy: {accuracy:.2f}")


Found 3371 images belonging to 4 classes.
Found 840 images belonging to 4 classes.
Epoch 1/10
106/106 [==============================] - 308s 3s/step - loss: 1.3543 - accuracy: 0.3664 - val_loss: 1.1533 - val_accuracy: 0.4071
Epoch 2/10
106/106 [==============================] - 309s 3s/step - loss: 0.9679 - accuracy: 0.5188 - val_loss: 0.8740 - val_accuracy: 0.5274
Epoch 3/10
106/106 [==============================] - 314s 3s/step - loss: 0.8247 - accuracy: 0.5731 - val_loss: 0.8421 - val_accuracy: 0.5512
Epoch 4/10
106/106 [==============================] - 303s 3s/step - loss: 0.7989 - accuracy: 0.5957 - val_loss: 0.8255 - val_accuracy: 0.5500
Epoch 5/10
106/106 [==============================] - 301s 3s/step - loss: 0.6749 - accuracy: 0.6719 - val_loss: 0.7922 - val_accuracy: 0.6048
Epoch 6/10
106/106 [==============================] - 348s 3s/step - loss: 0.6542 - accuracy: 0.6968 - val_loss: 0.9217 - val_accuracy: 0.5631
Epoch 7/10
106/106 [==============================] - 343s 

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Temporary base directory for the dataset
temp_base_dir = r"C:\Users\91790\OneDrive\Desktop\BHAVATHARINI\OCULAR DISEASE\Temp"

# Data generator with data augmentation and validation split
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Training dataset from specified subfolders
train_generator = datagen.flow_from_directory(
    temp_base_dir,
    target_size=(224, 224),  # VGG-like networks use 224x224 images
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

# Validation dataset from specified subfolders
val_generator = datagen.flow_from_directory(
    temp_base_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Define a simplified VGG-like architecture
simple_vgg = models.Sequential([
    layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
    layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(256, (3, 3), padding='same', activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_generator.num_classes, activation='softmax')  # Output for categorical classes
])

# Compile the model
simple_vgg.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
simple_vgg.fit(
    train_generator,
    epochs=5,
    validation_data=val_generator
)

# Evaluate the model
loss, accuracy = simple_vgg.evaluate(val_generator)
print(f"Validation accuracy: {accuracy:.2f}")


Found 3371 images belonging to 4 classes.
Found 840 images belonging to 4 classes.
Epoch 1/5
106/106 [==============================] - 425s 4s/step - loss: 1.0506 - accuracy: 0.5013 - val_loss: 0.8968 - val_accuracy: 0.5155
Epoch 2/5
106/106 [==============================] - 393s 4s/step - loss: 0.7459 - accuracy: 0.6253 - val_loss: 0.9203 - val_accuracy: 0.5274
Epoch 3/5
106/106 [==============================] - 523s 5s/step - loss: 0.6010 - accuracy: 0.7048 - val_loss: 1.3219 - val_accuracy: 0.5393
Epoch 4/5
106/106 [==============================] - 650s 6s/step - loss: 0.5530 - accuracy: 0.7419 - val_loss: 0.9299 - val_accuracy: 0.5393
Epoch 5/5
27/27 [==============================] - 60s 2s/step - loss: 1.0165 - accuracy: 0.5548
Validation accuracy: 0.55
